#### 상관계수

- 피어슨 상관계수
    - 두 변수 간의 선형 관계 측정하는 통계적 지표
    

In [ ]:
# 데이터 생성
import numpy as np
import scipy.stats as stats

x = np.array([10, 20, 30, 40, 50])
y = np.array([5, 15, 25, 35, 48])


# 피어슨 상관계수와 p-value 계산
corr_coeff, p_value = stats.pearsonr(x, y)

print(corr_coeff)
print(np.round(p_value, 4))

# 높은 상관 관계가 존재하고 유의수준 5%에서 통계적 유의성 존재

0.9984018457335851
0.0001


- 다중 선형 회귀

In [11]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

# 데이터 로드
df_iris = load_iris()

# 데이터 프레임 변경
iris = pd.DataFrame(data=df_iris.data, columns=df_iris.feature_names)
iris.columns = ['Sepal_Length', 'Sepal_Width', 'Petal_Length','Petal_Width'] # 컬럼 변경시

# 타겟 클래스 추가
iris['species'] = df_iris.target

# 클래스 라벨 실제 이름 변환
iris['species'] = iris['species'].map({0: 'setosa', 1:'versicolor', 2: 'virginica'})

#### 방법 1 Formula API

In [ ]:
# 방법 1 Formula API

import statsmodels.api as sm
import statsmodels.formula.api as smf

model = smf.ols('Petal_Length ~ Petal_Width + Sepal_Length', data= iris).fit() # y(Petal_Length/종속변수) ~ x1,x2(Petal_Width, Sepal_Length / 독립변수) 형식 컬럼이름 맞춰야함
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           Petal_Length   R-squared:                       0.949
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     1354.
Date:                Tue, 17 Jun 2025   Prob (F-statistic):           2.01e-95
Time:                        15:34:04   Log-Likelihood:                -75.090
No. Observations:                 150   AIC:                             156.2
Df Residuals:                     147   BIC:                             165.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.5071      0.337     -4.473   

In [ ]:
model = smf.glm('Petal_Length ~ Petal_Width + Sepal_Length', family = sm.families.Gaussian(), data=iris).fit()


# family = sm.families.Gaussian()는 다중 선형회귀 모형을 의미

# smf.ols는 smf.glm에 비해 선형회귀에 특화된 가설 검정 결과 제공


In [ ]:
# 독립 변수로 범주형 변수가 존재하는 경우 C()로 범주형 변수 명시

import statsmodels.api as sm
import statsmodels.formula.api as smf

model = smf.ols('Petal_Length ~ Petal_Width + Sepal_Length + C(species)', data=iris).fit()

#### 방법 2: 행렬 활용

In [22]:
import statsmodels.api as sm

X = iris[['Petal_Width', 'Sepal_Length']]
y = iris['Petal_Length']

X = sm.add_constant(X) # 상수항 즉 절편 추가

# 다중회귀 분석 모델 적합 
model = sm.OLS(y, X).fit()

# 회귀계수 출력
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           Petal_Length   R-squared:                       0.949
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     1354.
Date:                Tue, 17 Jun 2025   Prob (F-statistic):           2.01e-95
Time:                        15:48:11   Log-Likelihood:                -75.090
No. Observations:                 150   AIC:                             156.2
Df Residuals:                     147   BIC:                             165.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -1.5071      0.337     -4.473   

In [24]:
model = smf.glm("Petal_Length ~ Petal_Width + Sepal_Length", family = sm.families.Gaussian(), data=iris).fit()

In [30]:
# 독립변수로 범주형 변수가 존재하는 경우, 사전에 더미인코딩 적용

model = smf.glm("Petal_Length ~ Petal_Width + Sepal_Length", family = sm.families.Gaussian(), data=iris).fit()

# 독립변수 (Petal_Width, Sepal_Length) + 범주형 변수 species 추가
X = iris[['Petal_Width', 'Sepal_Length', 'species']]

# 범주형 변수
X = pd.get_dummies(X, columns=["species"], drop_first=True)
X = X.astype(float) # 해줘야함
y = iris['Petal_Length']
X = sm.add_constant(X) # 절편 추가
model = sm.OLS(y, X).fit()

In [31]:
# 회귀계수에 따른 모델식

coefficients = model.params[1:]
print('회귀계수가 가장 큰 변수', coefficients.idxmax())

회귀계수가 가장 큰 변수 species_virginica


In [32]:
t_values = model.tvalues
print("t-values:" , t_values)

p_values = model.pvalues
print('p-values', p_values)

t-values: const                 -6.519822
Petal_Width            4.392751
Sepal_Length          12.191282
species_versicolor    13.566844
species_virginica     11.617036
dtype: float64
p-values const                 1.092583e-09
Petal_Width           2.147008e-05
Sepal_Length          5.616327e-24
species_versicolor    1.384126e-27
species_virginica     1.824160e-22
dtype: float64


In [33]:
# 신뢰 구간 추출

conf_intervals = model.conf_int()
print(conf_intervals)

                           0         1
const              -1.902029 -1.017105
Petal_Width         0.278559  0.734267
Sepal_Length        0.468146  0.649309
species_versicolor  1.479213  1.983701
species_virginica   1.912572  2.696783


In [34]:
# 유의수준 변경 
conf_intervals_90 = model.conf_int(alpha = 0.1) # 90% 신뢰구간
print(conf_intervals_90)

                           0         1
const              -1.830162 -1.088972
Petal_Width         0.315569  0.697258
Sepal_Length        0.482859  0.634596
species_versicolor  1.520184  1.942730
species_virginica   1.976261  2.633095


#### 더미 변수 함정

In [41]:
import pandas as pd
import statsmodels.api as sm

# 샘플 데이터 생성
data = {
    'color': ['red', 'blue','green', 'red', 'green', 'red', 'green', 'blue', 'green', 'red'],
    'size': [1,2,3,1,3,5,9,2,9,10],
    'price': [10,20,30,10,30,55,29,10,25,12]
}
df=pd.DataFrame(data)

In [42]:
# 범주형 변수 더미 코딩 (drop=first = True로 기준 범수 제거) -> 다중공선성 제거
df_dummies = pd.get_dummies(df, columns=['color'], drop_first = True)
print(df_dummies)

   size  price  color_green  color_red
0     1     10        False       True
1     2     20        False      False
2     3     30         True      False
3     1     10        False       True
4     3     30         True      False
5     5     55        False       True
6     9     29         True      False
7     2     10        False      False
8     9     25         True      False
9    10     12        False       True


In [43]:
# 종속 변수와 독립 변수 설정
X = df_dummies[['size', 'color_green', 'color_red']]
y = df_dummies['price']

X = X.astype(float)
y = y.astype(float)

# 상수항 추가
X = sm.add_constant(X)

# 다중회귀모델 적합
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                 -0.280
Method:                 Least Squares   F-statistic:                    0.3430
Date:                Tue, 17 Jun 2025   Prob (F-statistic):              0.796
Time:                        16:49:20   Log-Likelihood:                -39.360
No. Observations:                  10   AIC:                             86.72
Df Residuals:                       6   BIC:                             87.93
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          14.3994     11.800      1.220      

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [44]:
# smf()를 활용
# smf()는 C(범주형 변수)로 설정할 경우 자동으로 더미 인코딩 적용ㅇ

import statsmodels.formula.api as smf

# 회귀 분석 공식 설정
formula = 'price ~ size + C(color)'

# 회귀 모델 적합
model = smf.ols(formula, data=df).fit()

# 모델 요약 출력
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                 -0.280
Method:                 Least Squares   F-statistic:                    0.3430
Date:                Tue, 17 Jun 2025   Prob (F-statistic):              0.796
Time:                        16:51:36   Log-Likelihood:                -39.360
No. Observations:                  10   AIC:                             86.72
Df Residuals:                       6   BIC:                             87.93
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            14.3994     11.80

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


#### 모델 평가
1. 모델 평가 지표
    1. 결정계수(R-squared)
        - 회귀 직선의 성능을 평가하는 지표

In [46]:
# 결정계수

print('R-squared:', np.round(model.rsquared, 2))

R-squared: 0.15


In [ ]:
# 수정 결정계수: 모델 복잡도를 고려한 지표 / 일반적으로 모델에 들어있는 독립변수 개수 p가 늘어나면 r-squared도 높아지는 경향을 보임
print(np.round(model.rsquared_adj, 2))

-0.28


In [49]:
# f-statistic: f통계량은 회귀 모델의 유의성을 평가하는 지표
print('f-statistic', np.round(model.fvalue, 4))
print('porb (f-statistic) F통계량에대한 p_value', np.round(model.f_pvalue, 4))

f-statistic 0.343
porb (f-statistic) F통계량에대한 p_value 0.7958


In [50]:
# AIC, BIC

print('AIC', np.round(model.aic,2))
print('BIC', np.round(model.bic,2))

AIC 86.72
BIC 87.93


2) 모델 비교하기 - model1 vs model2

In [ ]:
# f-검정

import statsmodels.api as sm
from statsmodels.formula.api import ols

model1 = ols('Petal_Length ~ Petal_Width', data=iris).fit()
model2 = ols('Petal_Length ~ Petal_Width + Sepal_Length +Sepal_Width', data=iris).fit()

table = sm.stats.anova_lm(model1, model2) #anova
print(table)

# reduced 모델이 첫번째, full모델이 두번째

   df_resid        ssr  df_diff    ss_diff          F        Pr(>F)
0     148.0  33.844753      0.0        NaN        NaN           NaN
1     146.0  14.852948      2.0  18.991805  93.341859  7.752746e-27


In [51]:
iris.head()

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


#### 예측